In [ ]:
"""
Created on Mon Apr 08 16:22 2024

This script is to make a timeseries of runs that have crossed the viability limit + compare to hydrofracturing

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os

In [ ]:
#sns.set_context('poster')
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
geoyear = 2100

if geoyear == 2100:
    weird_isf = [57,47,32,36,41]
elif geoyear == 2150:
    weird_isf = [57,72,35,19,36,41,26]

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/ElmerIce_'+str(geoyear)+'/'
file_isf_orig = xr.open_dataset(inputpath_mask+'ElmerIce_4km_'+str(geoyear)+'isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]
sorted_isf_rignot = [x for x in sorted_isf_rignot if x not in weird_isf]
file_isf = file_isf_nonnan.sel(Nisf=sorted_isf_rignot)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS_ElmerIcegeo'+str(geoyear)+'.nc')

In [ ]:
### Colorbar:
cmap = mpl.cm.YlOrRd # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

alpha = 0.8
cmap_with_alpha = cmap(np.arange(cmap_new.N))
cmap_with_alpha[:, -1] = alpha
cmap_new2 = mpl.colors.ListedColormap(cmap_with_alpha)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new2.N)

In [ ]:
rgba = cmap(0.33)
print(rgba)

Air temperature

In [ ]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/CMIP_TEMP/'
#mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-ESM2-1','GISS_E2-1-H',
#            'CESM2-WACCM','CESM2','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0','MPI-ESM1-2-HR',
#            'GFDL-CM4','GFDL-ESM4','UKESM1-0-LL']

mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #,

tas_mod_list = []

for mod in mod_list:
    print(mod)
    if mod not in ['GISS_E2-1-H']:
        tas_scen_list = []
        for scen in ['historical','ssp585']:
            if (mod == 'GFDL-CM4') and (scen == 'ssp126'):
                tas_mod_scen['tas'] = tas_mod_scen['tas']*np.nan
            else:
                tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_fldmean_ymean.nc')
                #tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_ymean_fldmeansouthof60.nc')
                tas_mod_scen['time'] = tas_mod_scen.time.dt.year
            tas_out = tas_mod_scen['tas'].squeeze(drop=True)
            if 'height' in tas_out.coords:
                tas_out = tas_out.drop('height')
            tas_scen_list.append(tas_out.assign_coords({'scenario':scen}))

        tas_all_scen = xr.concat(tas_scen_list,dim='scenario')
        tas_mod_list.append(tas_all_scen.assign_coords({'model':mod}))
        
tas_all_mod = xr.concat(tas_mod_list, dim='model') 

In [ ]:
tas_all_mod_ano = tas_all_mod - tas_all_mod.sel(scenario='historical',time=range(1850,1901)).mean('time')

Hydrofracturing limits

In [ ]:
hf_limit = xr.open_dataset(inputpath_data + 'hydrofracturing_limits_new.nc')['hydrofrac_limit']

PREPARE THE DATA

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})
sens_weights = sens_weights.drop_sel(model='GISS-E2-1-H')

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #'GISS-E2-1-H',

In [ ]:
#bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
#total_weights = bay_weights * sens_weights
total_weights_2300 = bay_weights_2300 * sens_weights

In [ ]:
# CALVING = 0
#mass_balance_weighted_yy = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(time=range(geoyear,2101)).weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(time=range(geoyear,2300)).sel(model=model_2300, Nisf=sorted_isf_rignot).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
#mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101)).weighted(bay_weights * sens_weights)
#mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
#weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])

In [ ]:
#unviable_times = weighted_quantiles > 0
unviable_times_2300 = weighted_quantiles_2300 > 0


In [ ]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2298,geoyear-1,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2299)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=geoyear)), geoyear)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')

In [ ]:
aslikelyasnot = limit_max_full_2300.sel(quantile=0.66)

In [ ]:
aslikelyasnot.where((aslikelyasnot < 1950) & (aslikelyasnot > 1850), drop=True)

In [ ]:
count_unviable_isf_2300.sel(scenario='ssp585').sel(time=2297).sel(quantile=[0.1,0.33,0.66,0.9])

In [ ]:
count_unviable_isf_2300.sel(time=2150).sel(quantile=[0.1,0.33,0.66,0.9])

In [ ]:
aslikelyasnot.sel(Nisf=25)

In [ ]:
file_isf['isf_name'].sel(Nisf=25)

In [ ]:
limit_max_full_2300.sel(Nisf=32,quantile=[0.33,0.1])

Do the same for hydrofracturing

In [ ]:
limit_max_full_2300

In [ ]:
count_hflimit_isf_2300 = (unviable_times_2300.time >= hf_limit).sum('Nisf')

Try to find some limit

In [ ]:
limit_of_int = limit_max_full_2300.sel(quantile=0.33)
time_of_int = weighted_mean_tas.time.where((weighted_mean_tas > 4) & (weighted_mean_tas <= 5), drop=True)
isf_4_5 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

In [ ]:
time_of_int = weighted_mean_tas.sel(scenario='ssp585').time.where((weighted_mean_tas.sel(scenario='ssp585') > 5) & (weighted_mean_tas.sel(scenario='ssp585') <= 6), drop=True)
isf_5_6 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

In [ ]:
time_of_int = weighted_mean_tas.sel(scenario='ssp585').time.where((weighted_mean_tas.sel(scenario='ssp585') > 6) & (weighted_mean_tas.sel(scenario='ssp585') <= 7), drop=True)
isf_6_7 = limit_of_int.where((limit_of_int >= time_of_int.min()) & (limit_of_int <= time_of_int.max()), drop=True)

In [ ]:
for kisf in isf_4_5.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
for kisf in isf_5_6.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
for kisf in isf_6_7.Nisf:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
(weighted_mean_tas.sel(scenario='ssp585') > 4).time.where((weighted_mean_tas.sel(scenario='ssp585') > 4) & (weighted_mean_tas.sel(scenario='ssp585') <= 5), drop=True)

In [ ]:
weighted_tas_2100 = tas_all_mod_ano.weighted(bay_weights * sens_weights) 
weighted_mean_tas_2100 = weighted_tas_2100.mean(['Nisf','model','param','m'])
weighted_tas_std_2100 = weighted_tas_2100.std(['Nisf','model','param','m'])


In [ ]:
weighted_tas = tas_all_mod_ano.sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)) 
weighted_mean_tas = weighted_tas.mean(['Nisf','model','param','m'])
weighted_tas_std = weighted_tas.std(['Nisf','model','param','m'])


In [ ]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen,time=range(1850,2101)), alpha=0.5)

weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen3,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen3,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen3,time=range(1850,2101)), alpha=0.5)

weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)).plot()
plt.fill_between(x=range(1850,2101),y1=weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)) - weighted_tas_std_2100.sel(scenario=scen2,time=range(1850,2101)), 
                 y2=weighted_mean_tas_2100.sel(scenario=scen2,time=range(1850,2101)) + weighted_tas_std_2100.sel(scenario=scen2,time=range(1850,2101)), alpha=0.5)

sns.despine()
plt.title('')
plt.xlabel('Time')
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
f.savefig(plot_path + 'tas_evolution_2100.png',dpi=250)
#plt.axvline(x=2075,linestyle='--')
#plt.axvline(x=2180,linestyle='--')

In [ ]:
scen = 'ssp126'
scen2 = 'ssp585'
scen3 = 'ssp245'

f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

weighted_mean_tas.sel(scenario=scen).plot(color='pink')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen) - weighted_tas_std.sel(scenario=scen), 
                 y2=weighted_mean_tas.sel(scenario=scen) + weighted_tas_std.sel(scenario=scen), alpha=0.5, color='pink')

weighted_mean_tas.sel(scenario=scen3).plot(color='orchid')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen3) - weighted_tas_std.sel(scenario=scen3), 
                 y2=weighted_mean_tas.sel(scenario=scen3) + weighted_tas_std.sel(scenario=scen3), alpha=0.5, color='orchid')

weighted_mean_tas.sel(scenario=scen2).plot(color='darkmagenta')
plt.fill_between(x=range(1850,2301),y1=weighted_mean_tas.sel(scenario=scen2) - weighted_tas_std.sel(scenario=scen2), 
                 y2=weighted_mean_tas.sel(scenario=scen2) + weighted_tas_std.sel(scenario=scen2), alpha=0.5, color='darkmagenta')

plt.axhline(y=4.7,xmin=(2085-2014)/(2300-2014),xmax=1,color='k',linestyle='--')
#plt.axhline(y=9.8,xmin=(2185-2014)/(2300-2014),xmax=1,color='k',linestyle='--')
plt.axvline(x=2085,ymax=4.7/14.5,color='k',linestyle='--')
#plt.axvline(x=2185,ymax=9.8/14.5,color='k',linestyle='--')


#plt.text(2200,4.7,'4.7$\pm$0.6')

sns.despine()
plt.title('')
plt.xlim(2014,2300)
plt.ylim(0,14.5)
plt.xlabel('Time')
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
f.savefig(plot_path + 'tas_evolution.png',dpi=250)
#f.savefig(plot_path + 'tas_evolution_southof60.png',dpi=250)


#plt.axvline(x=2075,linestyle='--')
#plt.axvline(x=2180,linestyle='--')

In [ ]:
weighted_mean_tas.sel(time=range(2100,2298))

In [ ]:
count_unviable_isf_2300.sel(quantile=0.33,time=range(2100,2298))

In [ ]:
f = plt.figure()
f.set_size_inches(8.25/1.5, 8.25/1.5)

plt.scatter(count_unviable_isf_2300.sel(quantile=0.33,time=range(geoyear,2298)), weighted_mean_tas.sel(scenario='ssp585',time=range(geoyear,2298)),s=10,alpha=0.3,color='darkmagenta')

plt.axhline(y=4.7,color='k',linestyle='--')
#plt.axhline(y=9.8,color='k',linestyle='--')
plt.text(20,4.9,'4.7$\pm$0.6°C')
#plt.text(20,10,'9.8$\pm$1.8°C')

sns.despine()
plt.title('')
plt.xlim(0,52)
plt.ylim(0,14.5)
plt.ylabel('Global surface air temperature anomaly w.r.t. 1850-1900 [K]')
plt.xlabel('Number of likely non-viable ice shelves')
#plt.axhline(y=4.5)
f.savefig(plot_path + 'tas_nonviable_scatter_'+str(geoyear)+'.png',dpi=250)
#f.savefig(plot_path + 'tas_southof60_nonviable_scatter.png',dpi=250)

In [ ]:
count_unviable_isf_2300.sel(time=2255, scenario='ssp126', quantile=0.9)

In [ ]:

limit_max_full_2300.sel(quantile=0.33,scenario='ssp126').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp126') < 2298).dropna('Nisf')

In [ ]:
date_of_int = 2084
limit_max_full_2300.sel(quantile=0.33,scenario='ssp585').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') < date_of_int).dropna('Nisf')

In [ ]:
date_of_int = 2184
limit_max_full_2300.sel(quantile=0.33,scenario='ssp585').where(limit_max_full_2300.sel(quantile=0.33,scenario='ssp585') < date_of_int).dropna('Nisf')

In [ ]:
26/64

In [ ]:
(35-3)/64

In [ ]:
diff_scen = ( (unviable_times.time >= limit_max_full).sel(scenario='ssp126').sel(time=range(1850,2000)).astype(int) - (unviable_times.time >= limit_max_full).sel(scenario='ssp585').sel(time=range(1850,2000)).astype(int))

In [ ]:
unviable_times.sel(Nisf=25,quantile=0.33,time=1850)

In [ ]:
limit_max_full.sel(Nisf=42,quantile=0.9)

In [ ]:
diff_scen.where(diff_scen != 0, drop=True).sel(quantile=0.9,Nisf=42)

In [ ]:
count_unviable_isf_2300.sel(quantile=qq_list[n-1],scenario=scen,time=range(1850,2014))

In [ ]:
scen='ssp585'

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
#f.set_size_inches(8.25, 8.25)
f.set_size_inches(8.25*0.75, 8.25/2*0.75) #*1.25

ax={}

leg_hdl = []

i = 0

#plt.fill_between(x=range(1850,2014), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(1850,2014)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(1850,2014)), color='grey',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp585',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp585',time=range(2014,2299)), color='deeppink',alpha=0.2)
#plt.fill_between(x=range(2014,2299), y1=count_unviable_isf_2300.sel(quantile=0.33,scenario='ssp126',time=range(2014,2299)), y2=count_unviable_isf_2300.sel(quantile=0.66,scenario='ssp126',time=range(2014,2299)), color='violet',alpha=0.2)

#qq_list = [1,0.9,0.66,0.33,0.1,0]
qq_list = [0,0.1,0.33,0.66,0.9]
col_tot = 4

for n in range(len(qq_list)-1,0,-1):
    if qq_list[n] == 0.1:
        plt.fill_between(x=range(geoyear,2298),y1=0,
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n],time=range(geoyear,2298)), color=cmap_new2(1-0.0001))
        plt.fill_between(x=range(geoyear,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],time=range(geoyear,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],time=range(geoyear,2298)), color=cmap_new2(1 - n/col_tot))
    elif qq_list[n] == 0.9:
        plt.fill_between(x=range(geoyear,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],time=range(geoyear,2298)), color=cmap_new2(1-n/col_tot),
                                            y2=64)
        
    else:
        plt.fill_between(x=range(geoyear,2298),y1=count_unviable_isf_2300.sel(quantile=qq_list[n],time=range(geoyear,2298)),
                                            y2=count_unviable_isf_2300.sel(quantile=qq_list[n+1],time=range(geoyear,2298)), color=cmap_new2(1 - n/col_tot))

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)
plt.xlim(1850,2298)
plt.ylim(0,len(sorted_isf_rignot))
plt.ylabel('Number of non-viable ice shelves')
plt.xlabel('Time')
f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'nb_non_viable_isf_2300_withoutGISS_fill_'+scen+'_ElmerIcegeo'+str(geoyear)+'.pdf')